# Домашнее задание 2. По чуть-чуть про pandas, sklearn, kNN

В этом задании вы в миниатюре пройдете цикл решения задачи анализа данных от предобработки данных до построения модели и оценки качества. Вы будете решать задачу кредитного скоринга (вернет клиент кредит или нет) на примере небольшой выборки данных. 

В процессе решения задания вы поближе познакомитесь с библиотеками pandas, sklearn и тем, как работают классы в python, а также немного вспомните numpy, самостоятельно реализовав 1NN (это очень просто, потому что сам алгоритм очень простой).

Все задачи нужно выполнять исключительно самостоятельно! Если что-то непонятно, задавайте вопросы в чате. При обнаружении списывания ноль за задание ставится и тому, кто списал, и тому, у кого списали.

Также во всем задании запрещено использование циклов (исключение - второе бонусное задание)! Все операции необходимо выполнять средствами numpy, pandas и sklearn.

Разделение выборки на обучение и контроль производится случайно (если только вы не указали конкретный random_state), поэтому результаты при разных запусках ноутбука могут получаться разные.

Считывание данных уже сделано:

In [273]:
import numpy as np
import pandas as pd

In [274]:
tab = pd.read_csv("https://raw.githubusercontent.com/nadiinchi/iad2019/master/materials/scoring.csv")

Выведите первые несколько строчек датасета, посмотрите названия колонок:

In [275]:
tab.head(3)


,account,duration,credit_history,purpose,amount,savings,employment,installment_rate,guarantors,residence,...,job,maintenance_people,telephone,foreign,real_estate,life_insurance,car,housing_rent,sex,target
0,0,18,4,furniture,3780,1,1,3,1,2,...,3,1,1,yes,0,0,1,0,male,1
1,1,24,2,car_new,1246,1,1,4,1,2,...,1,1,0,yes,1,0,0,0,male,0
2,0,15,2,car_used,3029,1,3,2,1,2,...,2,1,0,yes,0,0,1,0,male,1


Сначала несколько вопросов просто так:
    
 __[0.5 балла]__ Какой процент клиентов моложе 30 лет (колонка age) снимает жилье (колонка housing_rent)? А какой процент клиентов старше 40 лет снимает жилье?

In [276]:
print(len(tab[(tab.housing_rent==1)&(tab.age < 30)])*100/len(tab[(tab.age<30)]), len(tab[(tab.housing_rent==1)&(tab.age > 40)])*100/len(tab[(tab.age>40)]))

31.6017316017316 9.67741935483871


 __[0.5 балла]__ У скольких иностранцев (foreign == 'yes') число кредитов (credits_number) больше 2?

In [277]:
len(tab.loc[(tab.foreign=='yes')&(tab.credits_number > 2)])

19

 __[0.5 балла]__ Выведите колонки job, life_insurance и guarantors для клиентов с номерами 105, 158, 306 и 336 одной таблицей (воспользуйтесь индексацией):

In [278]:
tab.iloc[[105, 158, 306, 336], tab.columns.get_indexer(['age', 'life_insurance', 'guarantors'])]

,age,life_insurance,guarantors
105,51,1,1
158,27,0,1
306,39,0,0
336,24,1,1


 __[0.5 балла]__ Выведите среднюю сумму кредита (колонка amount) в каждой категории (колонка purpose), воспользовавшись методом groupby:

In [279]:
tab.groupby('purpose').mean()['amount']

purpose
business               4405.098361
car_new                3156.796178
car_used               5690.403509
domestic_appliances    1307.875000
education              3276.258065
furniture              3312.677966
others                 8681.700000
repairs                2792.769231
retraining             1156.000000
television             2554.237762
Name: amount, dtype: float64

Теперь по существу: нужно подготовить данные к построению модели.

 __[0.5 балла]__ Как вы видите, не все данные являются числовыми. С признаком purpose мы работать не будем - удалите его из датафрейма:

In [280]:
tab = tab.drop('purpose', 1)
tab.head(3)

,account,duration,credit_history,amount,savings,employment,installment_rate,guarantors,residence,age,...,job,maintenance_people,telephone,foreign,real_estate,life_insurance,car,housing_rent,sex,target
0,0,18,4,3780,1,1,3,1,2,35,...,3,1,1,yes,0,0,1,0,male,1
1,1,24,2,1246,1,1,4,1,2,23,...,1,1,0,yes,1,0,0,0,male,0
2,0,15,2,3029,1,3,2,1,2,33,...,2,1,0,yes,0,0,1,0,male,1


 __[0.5 балла]__ Необходимо перекодировать оставшиеся строковые значения в числа (закодировать пол female->0 и male->1 и колонку foreign числами yes->1 и no->0). Сделайте это с помощью pandas (без циклов, конечно же):

In [281]:
tab['sex'] = tab['sex'].map({'female': 0, 'male': 1})
tab['foreign'] = tab['foreign'].map({'yes': 1, 'no': 0})
tab

,account,duration,credit_history,amount,savings,employment,installment_rate,guarantors,residence,age,...,job,maintenance_people,telephone,foreign,real_estate,life_insurance,car,housing_rent,sex,target
0,0,18,4,3780,1,1,3,1,2,35,...,3,1,1,1,0,0,1,0,1,1
1,1,24,2,1246,1,1,4,1,2,23,...,1,1,0,1,1,0,0,0,1,0
2,0,15,2,3029,1,3,2,1,2,33,...,2,1,0,1,0,0,1,0,1,1
3,0,36,3,4463,1,2,4,1,2,26,...,3,1,1,1,0,0,1,0,1,0
4,-1,27,2,3416,1,2,3,1,2,27,...,3,1,0,1,0,0,1,0,1,1
5,-1,24,3,1333,1,0,4,1,2,43,...,2,2,0,1,1,0,0,0,1,0
6,0,15,2,5324,3,4,1,1,4,35,...,2,1,0,1,0,0,0,0,0,1
7,1,30,4,4249,1,0,4,1,2,28,...,3,1,0,1,0,0,1,0,0,0
8,0,15,4,2788,1,3,2,0,3,24,...,2,1,0,1,0,0,1,0,0,1
9,1,12,2,1223,1,4,1,1,1,46,...,2,1,0,1,1,0,0,1,1,0


 __[0.5 балла]__ Целевая переменная хранится в колонке target (последняя колонка). Являются ли классы сбалансированными (одинаковое число 0 и 1)? Воспользуйтесь методом [value_counts](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html).

In [282]:
tab['target'].value_counts()[0] == tab['target'].value_counts()[1]

True

Количество 0 и 1 одинаково, поэтому сбалансированны

Выведем статистику датасета:

In [283]:
tab.describe()

,account,duration,credit_history,amount,savings,employment,installment_rate,guarantors,residence,age,...,job,maintenance_people,telephone,foreign,real_estate,life_insurance,car,housing_rent,sex,target
count,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,...,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000
mean,0.083333,21.878333,2.410000,3470.491667,1.200000,2.348333,2.988333,0.901667,2.831667,35.181667,...,1.906667,1.150000,0.391667,0.965000,0.268333,0.205000,0.343333,0.186667,0.598333,0.500000
std,0.915306,12.749161,1.098747,3090.052187,0.924531,1.208069,1.131517,0.298013,1.103945,11.416182,...,0.662334,0.357369,0.488530,0.183933,0.443462,0.404038,0.475218,0.389969,0.490644,0.500417
min,-1.000000,4.000000,0.000000,250.000000,0.000000,0.000000,1.000000,0.000000,1.000000,19.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-1.000000,12.000000,2.000000,1363.250000,1.000000,2.000000,2.000000,1.000000,2.000000,26.000000,...,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,18.000000,2.000000,2347.000000,1.000000,2.000000,3.000000,1.000000,3.000000,33.000000,...,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.500000
75%,1.000000,27.000000,4.000000,4318.750000,1.000000,3.000000,4.000000,1.000000,4.000000,41.000000,...,2.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000
max,2.000000,72.000000,4.000000,18424.000000,4.000000,4.000000,4.000000,1.000000,4.000000,75.000000,...,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


 __[0.5 балла]__ Проанализируйте таблицу. Есть ли признаки с пропущенными значениями? (ответ необходимо писать в этой же ячейке; к ответу сделайте пояснение)

In [284]:
print(tab.isnull().sum())


account               0
duration              0
credit_history        0
amount                0
savings               0
employment            0
installment_rate      0
guarantors            0
residence             0
age                   0
credits_number        0
job                   0
maintenance_people    0
telephone             0
foreign               0
real_estate           0
life_insurance        0
car                   0
housing_rent          0
sex                   0
target                0
dtype: int64


Команда складывает количество NaN по каждому признаку - видим, что они отсутствуют

 __[0.5 балла]__ Запишите в переменную X матрицу объекты-признаки (выделите все колонки признаков, кроме target), а в переменную y - вектор правильных ответов (колонка target). К обоим объектам припишите .values, чтобы "достать" numpy-матрицу из датафрейма: далее мы везде будем работать с numpy-матрицами.

In [285]:
X = tab.loc[:, tab.columns != 'target'].values
Y = tab['target'].values
print(X, Y)

[[ 0 18  4 ...  1  0  1]
 [ 1 24  2 ...  0  0  1]
 [ 0 15  2 ...  1  0  1]
 ...
 [ 0 24  4 ...  0  0  1]
 [ 0 12  2 ...  1  1  0]
 [ 0 24  4 ...  0  0  1]] [1 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 0 1 1 0 1 0 0 1 0 1 1 1 1 1 1 0 1 0 0 0
 0 0 0 0 0 1 1 0 1 0 1 0 1 1 1 1 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0
 0 1 1 1 1 0 0 1 1 1 0 0 1 1 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1 0 1 1
 0 0 0 1 0 0 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 1
 1 0 1 0 0 1 1 1 0 1 1 1 1 1 0 0 0 0 1 1 0 0 1 1 1 1 1 1 1 0 0 0 1 1 1 0 1
 1 1 0 0 0 1 1 0 0 1 1 0 1 0 0 1 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 1 1
 1 0 0 1 1 0 1 0 1 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0
 0 0 1 1 0 1 0 0 1 1 1 0 0 1 1 1 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 1 0 0 1 1 1
 0 1 0 0 1 1 0 1 1 0 0 1 0 0 0 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 0 0 1 0 1
 0 1 1 0 1 0 1 1 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1
 0 1 1 1 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 1 0 1 1 1 1 1 0 1 0 0 0 0 1
 1 0 1 1 0 1 0 0 0 

 __[0.5 балла]__ С помощью sklearn-функции train_test_split разделите данные на обучение и контроль (пропорцию оставьте по умолчанию). Ячейка ?train_test_split покажет документацию функции (с примерами использования), также можно загуглить документацию.

In [286]:
from sklearn.model_selection import train_test_split

In [287]:
?train_test_split

In [288]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

Выведите форму (shape) обучающей и контрольной выборок:

In [289]:
X_train.shape, X_test.shape

((450, 20), (150, 20))

 __[0.5 балла]__ Как вы видели выше, все колонки имеют разный масштаб, это будет мешать нам при запуске метрического классификатора. Отнормируйте данные с помощью StandardScaler (интерфейс такой же, как мы изучали на семинаре):

In [290]:
from sklearn.preprocessing import StandardScaler

In [291]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_trans = scaler.transform(X_train)
X_test_trans = scaler.transform(X_test)
X_train_trans, X_test_trans


C:\Users\shest\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\shest\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\shest\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


(array([[ 2.06624839, -1.26845252, -0.40531883, ..., -0.72486118,
         -0.45435043,  0.83170267],
        [-0.10874992, -1.42539605, -0.40531883, ..., -0.72486118,
          2.20094431, -1.20235277],
        [-1.19624907, -0.79762192, -0.40531883, ..., -0.72486118,
         -0.45435043,  0.83170267],
        ...,
        [-1.19624907,  0.14403929, -0.40531883, ..., -0.72486118,
          2.20094431, -1.20235277],
        [-0.10874992,  0.14403929, -0.40531883, ...,  1.3795745 ,
         -0.45435043,  0.83170267],
        [ 0.97874924, -0.32679131,  1.39166119, ..., -0.72486118,
         -0.45435043,  0.83170267]]),
 array([[-0.10874992, -0.79762192, -0.40531883, ...,  1.3795745 ,
          2.20094431, -1.20235277],
        [-1.19624907,  0.61486989, -2.20229884, ...,  1.3795745 ,
         -0.45435043,  0.83170267],
        [ 0.97874924, -0.79762192, -0.40531883, ...,  1.3795745 ,
         -0.45435043,  0.83170267],
        ...,
        [-1.19624907,  2.0273617 , -0.40531883, ..., -

 __[0.5 балла]__ Теперь обучите классификатор 1-NN из sklearn на обучающей выборке, сделайте предсказания отдельно для обучающей и для контрольной выборки, на каждой выборке посчитайте точность (accuracy) с помощью accuracy_score. При создании объекта KNeighborsClassifier необходимо указать число соседей 1 (по умолчанию оно больше). 

Интерфейс KNeighborsClassifier такой же, как мы изучали на семинаре, а accuracy-score принимает на вход вектор предсказаний и вектор правильных ответов.

In [292]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [293]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_trans, Y_train)
pred_train = knn.predict(X_train_trans)
pred_test = knn.predict(X_test_trans)
print(accuracy_score(pred_train, Y_train), accuracy_score(pred_test, Y_test)),

print(pred_test)

1.0 0.5533333333333333
[1 1 0 1 1 0 1 0 1 0 1 0 0 1 0 0 0 1 0 1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 1 1 1
 0 0 0 0 1 0 1 1 1 0 1 0 1 1 0 0 1 0 0 0 1 1 1 1 1 0 1 1 1 0 1 0 0 0 1 0 1
 0 0 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 1 1 0 1 1 0 1 1 1 0
 0 1 1 1 1 0 1 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 1 1 1 0 1 1 1 0
 0 0]


 __[4 балла,  включая my_accuracy и тестирование реализации]__ Наконец, реализуйте свой 1NN, воспользовавшись шаблоном ниже. Ваш класс должен уметь работать в двух режимах: в евклидовой метрикой и с манхэттанской метрикой (формулы метрик ниже). Напоминаем принцип работы 1NN: чтобы сделать предсказание для объекта, он выбирает ближайший объект из обучения и возвращает такой же класс, как у него.
* в методе \__init\__ необходимо запомнить, с какой метрикой мы работаем
* в методе fit нужно запомнить выборку - больше при обучении kNN ничего делать не нужно
* в методе predict нужно сделать три шага:
    * сначала построить матрицу расстояний между контрольными и обучающими объектами. Это умеет автоматически делать функция cdist, но за ее использование штраф 1.5 балла - лучше посчитать матрицу расстояний самостоятельно с помощью "чистого" numpy
    * затем взять argmin по строкам - получится вектор индексов ближайших обучающих объектов для каждого контрольного объекта
    * выбрать компоненты y_train, отвечающие компонентам вектора индексов. Иными словами, вернуть те же метки классов, что у ближайших обучающих объектов.
    
Формулы метрик (расстояний):
* евклидова: $$\rho(x, y) = \sqrt{\sum_{i=1}^d (x_i - y_i)^2}$$
* манхэттанская: $$\rho(x, y) = \sum_{i=1}^d |x_i - y_i|$$

Модуль в numpy вычисляется как np.abs.

In [294]:
from scipy.spatial.distance import cdist # за использование этой функции штраф 1.5 балла :)

In [295]:
class MyKNN:
    def __init__(self, metric="euclid"):
        self.metric = metric
        
    def fit(self, X_train, Y_train):
        self.X_train = X_train
        self.Y_train = Y_train
        
    def predict(self, X_test):
        self.X_test = X_test
        if self.metric == 'euclid':
            distance = np.sqrt((self.X_train**2).sum(axis=1)[:, np.newaxis] 
                + ((self.X_test**2).sum(axis=1)[np.newaxis, :]) - 2*self.X_train.dot(X_test.transpose()))
        
        elif self.metric == 'manhattan':
            distance = np.abs(self.X_train[:, np.newaxis] - X_test).sum(axis = -1)
            
        index = np.argmin(distance, axis = 0)
        pred_train = Y_train[index]
        return pred_train
    

Реализуйте также функцию вычисления точности - доли правильных ответов:

In [296]:
def my_accuracy(pred_train, Y_test):
    acc_score = len(pred_train[pred_train == Y_test]) / len(Y_test)
    return acc_score


Пришло время запустить ваш классификатор и вычислить точность с помощью вашей функции. Обучите классификатор, сделайте предсказания для обучающей и контрольной выборок и посчитайте точность в обоих случаях. Сделайте это два раза - ддя евклидовой и манхэттанской метрик. Для евклидовой метрики цифры должны получиться точно такие же, как для реализации sklearn.

In [297]:
knn2 = MyKNN('euclid')
knn2.fit(X_train_trans, Y_train)
pred_test2 = knn2.predict(X_test_trans)
print(my_accuracy(pred_test2, Y_test))
print(pred_test2)

### видим, что для евклидовой метрики цифры действительно совпадают с предсказанием из sklearn

0.5533333333333333
[1 1 0 1 1 0 1 0 1 0 1 0 0 1 0 0 0 1 0 1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 1 1 1
 0 0 0 0 1 0 1 1 1 0 1 0 1 1 0 0 1 0 0 0 1 1 1 1 1 0 1 1 1 0 1 0 0 0 1 0 1
 0 0 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 1 1 0 1 1 0 1 1 1 0
 0 1 1 1 1 0 1 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 1 1 1 0 1 1 1 0
 0 0]


### Бонусные задания

#### Бонусное задание 1: реализация kNN (до 3 баллов)
Добавьте в ваш KNN параметр k (число соседей) и реализуйте k-NN, который возвращает самый популярный класс среди k ближайших соседей объекта. Если число ближайших объектов для разных классов совпадает, возвращайте класс с меньшим номером. За реализацию, работающую для любого числа классов - 3 балла (посмотрите функцию np.bincount), за реализацию, работающую только для двух классов - 2 балла. Не забудьте протестировать свой алгоритм :) В этом задании циклы использовать нельзя!


In [298]:
class knnk:
    def __init__(self, n_neighbors, metric="euclid"):
        self.metric = metric
        self.n_neighbors = n_neighbors
   
    def fit(self, X_train, Y_train):
        self.X_train = X_train
        self.Y_train = Y_train
        
    def predict(self, X_test):
        self.X_test = X_test
        if self.metric == 'euclid':
            distance = np.sqrt((self.X_train**2).sum(axis=1)[:, np.newaxis] + 
                    ((self.X_test**2).sum(axis=1)[np.newaxis, :]) - 2*self.X_train.dot(X_test.transpose()))
        elif self.metric == 'manhattan':
            distance = np.abs(self.X_train[:, np.newaxis] - X_test).sum(axis = -1)
        index = np.argsort(distance, axis = 0)[:self.n_neighbors]
        pred_train = Y_train[index]
        pred_train_k = scipy.stats.mode(pred_train)[0][0]
        return pred_train_k

In [299]:
knn3 = knnk(3)
knn3.fit(X_train_trans, Y_train)
pred_test2 = knn3.predict(X_test_trans)
print(my_accuracy(pred_test2, Y_test))


0.6


#### Бонусное задание 2: реализация CentroidClassifier (3 балла)
Реализуйте другой метрический классификатор - центроидный. Он работает чуть-чуть сложнее kNN, но зато гораздо эффективнее: на этапе обучения классификатор запоминает центр каждого класса ("средний" объект класса). На этапе предсказания классификатор возвращает тот класс, чей центр ближе. Не забудьте протестировать свой алгоритм :) Метрика - евклидова, для вычисления центра класса нужно найти среднее каждого признака по объектам этого класса. В этом задании можно использовать 1 цикл - цикл по классам при подсчете центров классов.

In [300]:
class MyCentroid:
    def __init__(self, metric="euclid"):
        self.metric = metric
    def fit(self, X_train, Y_train):
        self.X_train = X_train
        self.Y_train = Y_train
        df_prep = np.hstack((X_train, Y_train[:, np.newaxis]))
        df = pd.DataFrame(df_prep)
        mean_array = np.array(df.groupby(df.iloc[:, -1]).mean().iloc[:, :-1])
        self.mean_array = mean_array
        
    def predict(self, X_test):
        self.X_test = X_test
        distance = np.sqrt((self.mean_array**2).sum(axis=1)[:, np.newaxis] + 
                    ((self.X_test**2).sum(axis=1)[np.newaxis, :]) - 2*self.mean_array.dot(X_test.transpose()))
        pred_train = np.argmin(distance, axis=0)
        return pred_train

In [301]:
CentroidClassifier = MyCentroid()
CentroidClassifier.fit(X_train_trans, Y_train)
pred_test2 = CentroidClassifier.predict(X_test_trans)
print(my_accuracy(pred_test2, Y_test)), 
print(pred_test2)

0.6666666666666666
[0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 1 1 1 0 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 0
 0 1 1 1 1 0 0 0 0 0 1 1 0 1 0 0 1 0 1 0 1 1 1 0 1 0 0 0 1 0 1 0 0 1 1 0 0
 0 0 0 0 1 1 1 0 1 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 0 0 1 1 0 1 1 0 1 1 1 0
 0 1 1 0 1 0 1 1 0 1 0 1 0 0 1 0 0 1 1 0 0 1 0 0 1 1 1 0 0 1 1 1 0 1 0 1 0
 0 1]
